In [1]:
#pip install kafka-python

     -------------------------------------- 246.5/246.5 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from kafka import KafkaProducer
from time import sleep
import pandas as pd
import json
from datetime import datetime
from confluent_kafka import Producer

In [2]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'])

In [3]:
# Topic namw is quickstart-events.
#producer.send('kafka7', json.dumps('Việt Nam').encode("utf-8"))

In [4]:
now = datetime.now()
now

datetime.datetime(2023, 7, 19, 19, 53, 58, 274940)

In [5]:
current_now = now.strftime('%d/%m/%Y %H:%M:%S')
current_now

'19/07/2023 19:53:58'

In [11]:
pip install confluent-kafka

Note: you may need to restart the kernel to use updated packages.


In [6]:
for i in range(5):
	massege = 'Câu văn Việt Nam thử: {}'.format(str(datetime.now().time()))
	producer.send('kafka6', massege.encode("utf-8"))
	sleep(2)
	print(massege, i)
    

Câu văn Việt Nam thử: 19:54:06.877420 0
Câu văn Việt Nam thử: 19:54:08.895414 1
Câu văn Việt Nam thử: 19:54:10.907454 2
Câu văn Việt Nam thử: 19:54:12.915328 3
Câu văn Việt Nam thử: 19:54:14.918589 4


### Read test file

In [3]:
#df = pd.read_csv('data_test_pharse1.csv')
df = pd.read_csv('baochi.csv')
df.head()

,content
0,Joe Biden . Hôm nay ( tức ngày 5 tháng 6 . là ...
1,NỮ SINH NGHÈO ĐƯỢC 4 ĐẠI HỌC TUYỂN THẲNG\nĐược...
2,"Nhà hàng kinh doanh ngang nhiên "" xẻ thịt "" cô..."
3,Cả nhà ơi mẹ em vừa gọi bảo trên quê em có một...
4,Ngoài vụ ở Thái Nguyên tôi đã vạch trần hôm ...


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  40 non-null     object
dtypes: object(1)
memory usage: 448.0+ bytes


## Preprocessing sentence

In [10]:
pip install pyvi

Note: you may need to restart the kernel to use updated packages.


In [4]:
import nltk
from pyvi import ViTokenizer
nltk.download('punkt')
import re
import html

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
user_regex = r"(@\w{1,15})"
hashtag_regex = r"(#\w{1,})"
url_regex = r"((https?|ftp|file):\/{2,3})+([-\w+&@#\/%=~|$?!:,.]*)|(www.)+([-\w+&@#\/%=~|$?!:,.]*)"
email_regex = r"[\w.-]+@[\w.-]+\.[a-zA-Z]{1,}"

def html_unescape(s):
    return html.unescape(s)

# Clear data
def clean_text(text):
    text = re.sub(url_regex, '', text)     # Xoá URL
    text = re.sub(email_regex, '', text)   # Xoá email
    text = re.sub(user_regex, '', text)    # Xoá tên người dùng dạng @...
    text = text.replace('#', ' ')          # Thay #HTML
    # text = text.replace('\n', '.')  = ''
    text = html_unescape(text)             # Decode kí tự 
    if text == '':
        return None
    return text

# Tokenizer
def word_segmentation(sentence):
    segmented_sentence = ViTokenizer.tokenize(sentence)
    segmented_words = segmented_sentence.split()
    return ' '.join(segmented_words)


def preprocessing(sentence):
    sentence = clean_text(sentence)
    sentence = word_segmentation(sentence)
    return sentence

## Load model

In [6]:
from sklearn.preprocessing import FunctionTransformer
def get_text_data(x):
    return [record[0] for record in x]
transformer_text = FunctionTransformer(get_text_data)

In [7]:
import joblib

# Đường dẫn tới file model
model_path =  "C:\\Users\\ADMIN\\BigData\\models\\modelSVM_to_treaming.hdf5"

# Đọc model từ file
model = joblib.load(model_path)
# Đường dẫn tới tệp chứa pipeline đã lưu
pipeline_path = "C:\\Users\\ADMIN\\BigData\\models\\pipeline.pkl"

# Tải pipeline từ tệp
pipeline_load = joblib.load(pipeline_path)

### Send messages

In [10]:
def send_message(topic):
    producer = KafkaProducer(bootstrap_servers=['localhost:9092'])
    
    for i in range(10):
        sentence = preprocessing(df.loc[i, 'content'])
        sentence_numpy = pd.DataFrame([sentence]).to_numpy()
        sentence_transform = pipeline_load.transform(sentence_numpy)
        # Dự đoán
        prediction = model.predict(sentence_transform)
        
        record = {'sentence-raw': df.loc[i, 'content'],'sentences-clean': sentence, 'label': int(prediction[-1])}
        try:
            # send message to topic
            producer.send(topic, json.dumps(record, ensure_ascii=False).encode('utf-8'))
            print("Producing message")
        except Exception as e:
            print("Fail produce message: {}".format(str(e)))
        sleep(5)
            


topic = 'detection'
send_message(topic)
print('Done')

Producing message
Producing message
Producing message
Producing message
Producing message
Producing message
Producing message
Producing message
Producing message
Producing message
Done


In [24]:
producer.close()

In [38]:

for i in range(10):
    sentence = preprocessing(df.loc[i, 'content'])
    sentence = pd.DataFrame([sentence]).to_numpy()
    sentence = pipeline_load.transform(sentence)
    # Dự đoán
    prediction = model.predict(sentence)

    record = {'date-time': current_now, 'sentence-raw': df.loc[i, 'content'], 'label': prediction[-1]}
    print(record)
    print('__'*60)

{'date-time': '14/07/2023 12:01:52', 'sentence-raw': "Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã có thông tin ban đầu về vụ nổ súng,cướp tiệm vàng tại chợ Đông Ba nằm trên đường Trần Hưng Đạo (TP Huế, tỉnh Thừa Thiên - Huế). Thông Sài Gòn Giải Phóng, khoảng 12h30' ngày 31/7, một đối tượng sử dụng súng AK bất ngờ xông vào tiệm vàng Hoàng Đức và Thái Lợi (phía trước chợ Đông Ba) rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng. Sau đó, đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu Gia Hội, cách khu vực gây án khoảng 300m. Giám đốc Công an tỉnh Thừa Thiên – Huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm Công an tỉnh và Công an TP Huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ, khoanh vùng và ngăn không để người dân đi vào hiện trường. Hàng trăm tiểu thương trong chợ Đông Ba và người dân gần cầu Gia Hội được yêu cầu di chuyển khỏi hiện trường, đóng cửa nhà đề phòng đạn lạc. Tuy nhiên, thấy vàng bị n